In [23]:
#Initial state of the door is unknown
bel_x0_open = 0.5
bel_x0_closed = 0.5

In [24]:
#Intitialize the measurement model
p_zt_open_xt_open = 0.6
p_zt_open_xt_closed = 0.2
p_zt_closed_xt_open = 0.4
p_zt_closed_xt_closed = 0.8

In [25]:
#Initialize the action model
p_xt_open_ut_push_xt_1_open = 1
p_xt_closed_ut_push_xt_1_open = 0
p_xt_open_ut_push_xt_1_closed = 0.8
p_xt_closed_ut_push_xt_1_closed = 0.2

p_xt_open_ut_do_nothing_xt_1_open = 1
p_xt_closed_ut_do_nothing_xt_1_open = 0
p_xt_open_ut_do_nothing_xt_1_closed = 0
p_xt_closed_ut_do_nothing_xt_1_closed = 1

In [26]:
def prediction(action):
    global bel_x0_open,bel_x0_closed
    if(action=="do_nothing"):
        bel_bar_x1_open = (p_xt_open_ut_do_nothing_xt_1_open*bel_x0_open) + (p_xt_open_ut_do_nothing_xt_1_closed*bel_x0_closed)

        bel_bar_x1_closed = (p_xt_closed_ut_do_nothing_xt_1_open*bel_x0_open) + (p_xt_closed_ut_do_nothing_xt_1_closed*bel_x0_closed)
    if(action=="open"):
        bel_bar_x1_open = (p_xt_open_ut_push_xt_1_open*bel_x0_open) + (p_xt_open_ut_push_xt_1_closed*bel_x0_closed)

        bel_bar_x1_closed = (p_xt_closed_ut_push_xt_1_open*bel_x0_open) + (p_xt_closed_ut_push_xt_1_closed*bel_x0_closed)

    return bel_bar_x1_open,bel_bar_x1_closed

In [27]:
def correction(bel_bar_x1_open,bel_bar_x1_closed,measurement):
    global bel_x0_open,bel_x0_closed
    if(measurement=="open"):
        bel_x1_open = p_zt_open_xt_open*bel_bar_x1_open

        bel_x1_closed = p_zt_open_xt_closed*bel_bar_x1_closed

    if(measurement=="closed"):
        bel_x1_open = p_zt_closed_xt_open*bel_bar_x1_open

        bel_x1_closed = p_zt_closed_xt_closed*bel_bar_x1_closed

    norm = 1/(bel_x1_open+bel_x1_closed)

    bel_x1_open = norm*bel_x1_open

    bel_x1_closed = norm*bel_x1_closed

    bel_x0_open = bel_x1_open
    bel_x0_closed = bel_x1_closed
        
    return bel_x0_open,bel_x0_closed

In [28]:
def bayes_filter(action,measurement):
    global bel_x0_open,bel_x0_closed
    bel_bar_x1_open,bel_bar_x1_closed = prediction(action)
    bel_x0_open,bel_x0_closed = correction(bel_bar_x1_open,bel_bar_x1_closed,measurement)
    print("Probability the door is open is ",bel_x0_open)
    print("Probability the door is closed is ",bel_x0_closed)

In [29]:
def main():
    actions = ["do_nothing","open","do_nothing","open","do_nothing"]
    measurement = ["closed","closed","closed","open","open"]

    # actions = ["open","open","open","open","open"]
    # measurement = ["closed","closed","closed","closed","closed"]

    for i in range(0,5):
         print("Action",actions[i])
         print("Measurement",measurement[i])
         bayes_filter(actions[i],measurement[i])
         print("Iteration",i+1,"done\n")
   
if __name__ == "__main__":
    main()

Action do_nothing
Measurement closed
Probability the door is open is  0.3333333333333333
Probability the door is closed is  0.6666666666666666
Iteration 1 done

Action open
Measurement closed
Probability the door is open is  0.7647058823529411
Probability the door is closed is  0.23529411764705882
Iteration 2 done

Action do_nothing
Measurement closed
Probability the door is open is  0.6190476190476191
Probability the door is closed is  0.38095238095238093
Iteration 3 done

Action open
Measurement open
Probability the door is open is  0.9732441471571905
Probability the door is closed is  0.026755852842809368
Iteration 4 done

Action do_nothing
Measurement open
Probability the door is open is  0.9909194097616345
Probability the door is closed is  0.009080590238365497
Iteration 5 done

